Create code for startup animation (concentric pulse)

In [1]:
from typing import Dict, Tuple, List
from math import ceil

## Define mapping from multiplex row / col to physical layout

(Mapping from individual grids to global row col same as in model.c)

In [2]:
led_rc_to_pos: Dict[Tuple[int, int], Tuple[float, float]] = {}

In [3]:
# input
for row in range(5):
    for col in range(5):
        led_rc_to_pos[(row, col)] = (col, row)

In [4]:
# conv0
conv0_rc_to_pos = lambda r,c: (5.67 + c * 0.66, 0.83 + r * 0.66)
for i in range(15):
    led_rc_to_pos[(i % 5, 5 + i // 5)] = conv0_rc_to_pos(i % 4, i // 4)
led_rc_to_pos[(5, 0)] = conv0_rc_to_pos(3, 3)

In [5]:
# conv1
conv1_rc_to_pos = lambda r,c: (9 + c * 0.66, 1.5 + r * 0.66)
for i in range(7):
    led_rc_to_pos[(5, i + 1)] = conv1_rc_to_pos(i % 3, i // 3)
led_rc_to_pos[(0, 8)] = conv1_rc_to_pos(1, 2)
led_rc_to_pos[(1, 8)] = conv1_rc_to_pos(2, 2)

In [6]:
# conv2
conv2_rc_to_pos = lambda r,c: (11.8 + c * 0.66, 2.3 + r * 0.66)
for i in range(4):
    led_rc_to_pos[(2 + i, 8)] = conv2_rc_to_pos(i % 2, i //2)

In [7]:
# pool
for i in range(6):
    led_rc_to_pos[(8, i + 3)] = (7 + i * 0.3, 4.5)
    
for i in range(9):
    led_rc_to_pos[(6, i)] = (7 + (i + 6) * 0.3, 4.5)
    
led_rc_to_pos[(8, 2)] = (7 + 15 * 0.3, 4.5)

In [8]:
# out
led_rc_to_pos[(8, 1)] = (7.9, 5.5)
for i in range(9):
    led_rc_to_pos[(7, i)] = (7.9 + (i + 1) * 0.5, 5.5)

## Calculate distances to central point and bin LEDs into animation groups

Each group will have the same brightness for each frame

In [9]:
def dist(x: Tuple[float, float], y: Tuple[float, float]) -> float:
    return ((x[0] - y[0])**2 + (x[1] - y[1])**2)**0.5

In [10]:
center = (13,7.67) # bottom left
center = (3.8 / 0.6, 2.7 / 0.6) # center
max_dist = max(dist(center, x) for x in led_rc_to_pos.values())
anim_steps = ceil(max_dist) * 2

dist_leds: List[List[Tuple[int, int]]] = [[] for i in range(anim_steps + 1)]
for rc, pos in led_rc_to_pos.items():
    r = int(round(dist(pos, center) / max_dist * anim_steps))
    dist_leds[r].append(rc)

In [11]:
anim_steps

16

## Print C code
Copy this into startup_anim.c (where indicated by comment).

In [ ]:
c = 0
for i, d_leds in enumerate(dist_leds):
    if d_leds:
        print(f"pwm_val = brightness_sequence[(i + {len(dist_leds) - i - 1}) % sizeof(brightness_sequence)];")
        for row, col in d_leds:
            print(f"set_led_brightness({row}, {col}, pwm_val);")
            if c % 25 == 0:
                print("run_pwm_cycle();")
            c += 1